This notebook is created to analyse simple correlations between entailment measures and human intution. Simply specify the file you want to analyse and run the remaining code.

In [193]:
data_file_location = "../data/output/test.csv"

In [194]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import nltk
from nltk.corpus import wordnet as wn

from operations.composition import *
from operations.similarity_measures import *
from operations.logical_negation import *
from operations.worldly_context_creation import *
from operations.helpers import *

import pandas as pd
import matplotlib.pyplot as plt

import pickle
import csv
import random


from matplotlib.lines import Line2D 
from matplotlib.ticker import MaxNLocator

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/benjaminrodatz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [195]:
file = []
df = None

with open(data_file_location, newline='') as csvfile:
        # with open('df_i4_all_idneg.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            test = row
            file.append(row)

        df = pd.DataFrame(file[1:])
        df.columns = file[0]


In [196]:
# convert the rows after row 5 to numeric.
# It is assumed that these only contain numbers which can then be compared to the human rating.
# This cell prints all rows which have been found and will eventually be compared to the human rating.

df["MEANRATING"] = pd.to_numeric(df["MEANRATING"])
for col in df.columns[5:]:
    print(col)
    df[col] = pd.to_numeric(df[col])
    


4_baseline_kba
4_worldly_context_kba


In [197]:
# This cell calculates and prints the correlations. 
# There may be some filtering going on, which can be adapted to once need.
cols = []
filter_term = "hyp"

for col in df.columns:        
    if "" in col:
        cols.append(col)

correlations = df[cols].corrwith(df["MEANRATING"])

df[cols]

,NEGATED,ALTERNATIVE,RELATION,MEANRATING,SPLIT,4_baseline_kba,4_worldly_context_kba
0,goat,beard,free_association,1.30,train,0.216249,0.488370
1,alligator,teeth,free_association,1.50,test,-0.415380,0.222114
2,chair,office,hypernym,1.60,train,-0.222410,-0.074476
3,yacht,job,unrelated,1.45,train,-0.262463,0.517264
4,goose,ducks,distributional,4.10,test,-0.114244,0.425544
...,...,...,...,...,...,...,...
1226,trout,living,unrelated,1.00,test,-0.395522,0.365334
1227,jacket,ones,unrelated,1.00,train,0.135149,0.265429
1228,television,rediffusion,cohyponym,1.40,test,0.213306,0.365173
1229,corn,students,unrelated,1.20,test,-0.481939,-0.026603


In [198]:
means = df[cols].mean()

In [199]:
means[25:]

Series([], dtype: float64)

In [200]:
nodes = [('w', 'baseline', '$w_N$'), 
         ('wc', 'worldly_context', '$\\texttt{wc}_{w_N}$'), 
         ('neg', 'logical', '$\\neg$'),
         ('f1', 'framework1', '$CN_{word1}$'), 
         ('f3', 'framework3', '$CN_{word3}$'), 
         ('f4', 'framework4', '$CN_{word4}$')
         ]

In [201]:
for node1 in nodes:
    for node2 in nodes:
        if node1 == node2:
            continue
        name = node1[1]+"_"+node2[1]
#         print(name, name in means.keys())
        if (name in means.keys()):
#             print(name)
            score = round((means[name]), 3)
            print("('{0}', '{1}', {2}),".format(node1[0], node2[0], score))

In [202]:
for i in range(len(nodes)):
    print("\\\\")
    print(nodes[i][2], end = '')
    for j in range(len(nodes)):
        name = nodes[i][1]+"_"+nodes[j][1]
        if name not in means.keys():
            name = nodes[j][1]+"_"+nodes[i][1]
#         print(name, name in means.keys())
        score = round(means[name], 3)
        print(" & {0}".format(score), end = '')

\\
$w_N$

KeyError: 'baseline_baseline'